# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,1.45,60,0,3.09,US,1734283529
1,1,puerto natales,-51.7236,-72.4875,19.25,45,75,10.29,CL,1734283531
2,2,broken hill,-31.9500,141.4333,30.78,17,8,5.17,AU,1734283532
3,3,bonthe,7.5264,-12.5050,30.16,61,5,5.71,SL,1734283535
4,4,wailua homesteads,22.0669,-159.3780,22.44,94,75,4.63,US,1734283536


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot

def plot_city_humidity_map(city_data_df):

    return city_data_df.hvplot.points(
        x='Lng', 
        y='Lat',
        geo=True,
        color='blue', 
        alpha=0.6,
        size='Humidity', 
        hover_cols=['City', 'Humidity'], 
        tiles='OSM',
        frame_width = 800,
        frame_height = 600,
    )

# Call the function with your DataFrame
# plot_city_humidity_map(city_data_df)
humidity_map_plot = plot_city_humidity_map(city_data_df)

# Display the map
humidity_map_plot
 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
cities_narrow_lats_df = city_data_df[(city_data_df['Lat'] >= -19) & (city_data_df['Lat'] <= 40)]

# Drop any rows with null values
ideal_weather_city_df = cities_narrow_lats_df.dropna()

# Display sample data
ideal_weather_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,bonthe,7.5264,-12.5050,30.16,61,5,5.71,SL,1734283535
4,4,wailua homesteads,22.0669,-159.3780,22.44,94,75,4.63,US,1734283536
6,6,acarau,-2.8856,-40.1200,30.56,58,15,7.28,BR,1734283539
7,7,weno,7.4515,151.8468,26.97,89,75,4.63,FM,1734283540
13,13,timbuktu,16.7735,-3.0074,29.65,5,1,5.09,ML,1734283548
...,...,...,...,...,...,...,...,...,...,...
552,552,hilo,19.7297,-155.0900,20.62,82,0,3.60,US,1734284344
557,557,sinazongwe,-17.2614,27.4618,29.30,57,19,0.95,ZM,1734284053
558,558,lionel town,17.8101,-77.2406,29.99,58,7,1.66,JM,1734284352
560,560,lagunas,-5.2269,-75.6753,26.66,95,100,1.26,PE,1734284354


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_city_df[['City', 'Country','Lat', 'Lng', 'Humidity']]
hotel_df = hotel_df.copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:, 'Hotel Name'] = ''
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
3,bonthe,SL,7.5264,-12.5050,61,
4,wailua homesteads,US,22.0669,-159.3780,94,
6,acarau,BR,-2.8856,-40.1200,58,
7,weno,FM,7.4515,151.8468,89,
13,timbuktu,ML,16.7735,-3.0074,5,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000 
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address =requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bonthe - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
acarau - nearest hotel: No hotel found
weno - nearest hotel: High Tide Hotel
timbuktu - nearest hotel: Hotel La Maison
ceeldheer - nearest hotel: No hotel found
garbahaarrey - nearest hotel: No hotel found
kapa'a - nearest hotel: Pono Kai Resort
bengkulu - nearest hotel: Dena Hotel
darwin - nearest hotel: Mantra Pantanas Darwin
hithadhoo - nearest hotel: Pebbles Inn
chiras - nearest hotel: No hotel found
dikwa - nearest hotel: No hotel found
hasaki - nearest hotel: Choshi Plaza Hotel
tenkodogo - nearest hotel: Hôtel Lafi 2*
mazagao - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
laguna - nearest hotel: Holiday Inn Express & Suites
mucurici - nearest hotel: Pousada Vista do Balneário
shingu - nearest hotel: ホテルニューパレス
santiago de cao - nearest hotel: Hostal El Encanto
abu zabad - nearest hotel: No hotel found

,City,Country,Lat,Lng,Humidity,Hotel Name
3,bonthe,SL,7.5264,-12.5050,61,No hotel found
4,wailua homesteads,US,22.0669,-159.3780,94,Hilton Garden Inn Kauai Wailua Bay
6,acarau,BR,-2.8856,-40.1200,58,No hotel found
7,weno,FM,7.4515,151.8468,89,High Tide Hotel
13,timbuktu,ML,16.7735,-3.0074,5,Hotel La Maison
...,...,...,...,...,...,...
552,hilo,US,19.7297,-155.0900,82,Dolphin Bay Hotel
557,sinazongwe,ZM,-17.2614,27.4618,57,Lake View Lodge
558,lionel town,JM,17.8101,-77.2406,58,No hotel found
560,lagunas,PE,-5.2269,-75.6753,95,Hospedaje Eco


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points( "Lng", "Lat", geo = True, tiles = "OSM", frame_width = 800, frame_height = 600, size = "Lat", scale = 1, color = "City", hover_cols = ["Hotel Name", "Country"] )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)